In [1]:
### Analyze MOFA MODEL Results
### Investigate the specific top features of each factor
### Plot the normalized expression of the features as heatmap

#############################################
# Prerequisites - Load Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/mofa_analysis//lib/R/library"


‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.4; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::betwee

In [3]:
source('MS2_Plot_Config.r')

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


In [4]:
source('MS1_Functions.r')

###############################################
# Preqrequisites Configurations & Parameters

In [5]:
### Load the parameters that are set via the configuration files

In [6]:
### Load configurations file
global_configs = read.csv('configurations/Data_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/Data_Configs.csv'”


In [7]:
head(global_configs,2)

,parameter,value
,<chr>,<chr>
1,data_path,/lustre/groups/epigenereg01/workspace/projects/jove/input_data/
2,result_path,/lustre/groups/epigenereg01/workspace/projects/jove/results/


In [8]:
data_path = global_configs$value[global_configs$parameter == 'data_path']

In [9]:
data_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/input_data/"

In [10]:
result_path = global_configs$value[global_configs$parameter == 'result_path']

In [11]:
result_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/results/"

In [12]:
### Configuration parameters for plot (read and assign)

In [13]:
plot_configs =  read.csv('configurations/05_Feature_Analysis_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/05_Feature_Analysis_Configs.csv'”


In [14]:
head(plot_configs,2)

,configuration_name,mofa_result_name,factor,top_variable_thres,faceting_variable,type,plot_width,plot_height
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>
1,MI_v1,MI_v1_MOFA,Factor2,0.005,measurement,CD4.TCM,8.07,5


In [15]:
## Assign the values from the configuration file to the corresponding variables

In [16]:
mofa_name = plot_configs$mofa_result_name
data_name = plot_configs$configuration_name

factor_var = plot_configs$factor
top_variable_fraction = plot_configs$top_variable_thres
variable = plot_configs$faceting_variable

view_var = plot_configs$type

In [17]:
## Extract the specified plot size
width_par = plot_configs$plot_width  # 8.07 default
height_par = plot_configs$plot_height

In [18]:
### Alternative: Set configurations manually 

# Load Data 

## MOFA Input

In [19]:
### Load the normalized expression data that was used as input for the MOFA model

In [20]:
path = paste0(result_path, '/02_results/02_Combined_Data_',data_name,'_INTEGRATED.csv')
data_long = read.csv(path)
data_long$X = NULL
print(file.info(path)$mtime)

[1] "2023-12-20 22:19:38 CET"


In [21]:
head(data_long,2)

,sample_id,variable,value,type,gene
,<chr>,<chr>,<dbl>,<chr>,<chr>
1,k10,B.cell__ACTB,0.4307273,B.cell,ACTB
2,k11,B.cell__ACTB,-0.7009514,B.cell,ACTB


## MOFA Model

In [22]:
### Load the trained MOFA Model

In [23]:
model_name =  paste0("03_MOFA_MODEL_", mofa_name,'.hdf5')

In [24]:
outfile = file.path( paste0(result_path, '/03_results/',  model_name) )

In [25]:
outfile

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/results//03_results/03_MOFA_MODEL_MI_v1_MOFA.hdf5"

In [26]:
model <- load_model(outfile, verbose = TRUE)

Loading data...

Loading expectations for 2 nodes...

Loading model options...

Loading training options and statistics...

Assigning names to the different dimensions...

Re-ordering factors by their variance explained...

Doing quality control...

Checking views names...

Checking groups names...

Checking samples names...

Checking features names...

Checking dimensions...

Checking there are no features with complete missing values...

Checking sample covariates...

Checking expectations...

Checking for intercept factors...

Warning message in .quality_control(object, verbose = verbose):
“Factor(s) 5, 9, 18 are strongly correlated with the total number of expressed features for at least one of your omics. Such factors appear when there are differences in the total 'levels' between your samples, *sometimes* because of poor normalisation in the preprocessing steps.
”
Checking for highly correlated factors...



In [27]:
### Load the Factor Data of the Model

In [28]:
factors = read.csv(paste0(result_path, '/03_results/', '03_Factor_Data_', mofa_name, '.csv'))

In [29]:
head(factors,2)

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,⋯,Factor12,Factor13,Factor14,Factor15,Factor16,Factor17,Factor18,Factor19,Factor20,sample_id
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0.1201561,-1.6391856,-0.4479716,0.6821012,2.02569211,1.125137,0.2297734,-0.1414955,1.8673081,-0.1146884,⋯,-0.5759286,0.04916295,-0.2147718,0.06947807,0.11059354,0.3949888,-0.029702059,0.9720044,-0.25919841,k1
2,0.6551561,-0.9849971,0.3236092,0.2551028,-0.09817868,0.653027,0.2648814,-1.3251484,-0.1440528,0.3070476,⋯,-0.5571489,0.17017130,-0.2884835,0.13961347,0.04162613,-0.2465334,0.002811495,0.1450809,-0.05609239,k10


In [30]:
### Load the Weight Data of the Model

In [31]:
weight_data = read.csv(paste0(result_path, '/03_results/', '03_Weight_Data_', mofa_name, '.csv'))

## Sample Meta Data

In [32]:
## Load the sample meta-data that contains covariates that can be used to group the plot

In [33]:
sample_data = read.csv(paste0(data_path, 'Prepared_Sample_Meta_Data', '.csv'))
sample_data$X = NULL

In [34]:
head(sample_data,2)

,sample_id,measurement,classification,delta_ef_value,delta_ef_value_class,CK,CRP,tp_outcome,sc_counts
,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<int>
1,k1,Control,healthy,NA,NA,43,0.4,NA,NA
2,k10,Control,coronary_vessel_disease,NA,NA,NA,0.2,NA,3648632


# Downstream Analysis of generated model

## Extract and prepare data for plots

### Merge factors and sample data

In [35]:
### Combine the factor and the sample data

In [36]:
head(factors,2)

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,⋯,Factor12,Factor13,Factor14,Factor15,Factor16,Factor17,Factor18,Factor19,Factor20,sample_id
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,0.1201561,-1.6391856,-0.4479716,0.6821012,2.02569211,1.125137,0.2297734,-0.1414955,1.8673081,-0.1146884,⋯,-0.5759286,0.04916295,-0.2147718,0.06947807,0.11059354,0.3949888,-0.029702059,0.9720044,-0.25919841,k1
2,0.6551561,-0.9849971,0.3236092,0.2551028,-0.09817868,0.653027,0.2648814,-1.3251484,-0.1440528,0.3070476,⋯,-0.5571489,0.17017130,-0.2884835,0.13961347,0.04162613,-0.2465334,0.002811495,0.1450809,-0.05609239,k10


In [37]:
factors_long = melt(factors, id.vars = 'sample_id')

In [38]:
head(factors_long,2)

,sample_id,variable,value
,<chr>,<fct>,<dbl>
1,k1,Factor1,0.1201561
2,k10,Factor1,0.6551561


In [39]:
merged_data_long = merge(factors_long, sample_data, by.x = 'sample_id', by.y = 'sample_id')

In [40]:
head(merged_data_long,2)

,sample_id,variable,value,measurement,classification,delta_ef_value,delta_ef_value_class,CK,CRP,tp_outcome,sc_counts
,<chr>,<fct>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<int>
1,k1,Factor1,0.1201561,Control,healthy,NA,NA,43,0.4,NA,NA
2,k1,Factor3,-0.4479716,Control,healthy,NA,NA,43,0.4,NA,NA


### Prepare weight data

In [41]:
### Adjust the format of the feature factor weight data to long

In [42]:
feature_weights_long = melt(weight_data, id.vars = c('variable_name', 'type'))

In [43]:
feature_weights_long$view = feature_weights_long$type
feature_weights_long$gene = str_replace(feature_weights_long$variable_name, '.*__', '')

In [44]:
head(feature_weights_long,2)

,variable_name,type,variable,value,view,gene
,<chr>,<chr>,<fct>,<dbl>,<chr>,<chr>
1,B.cell__ACTB,B.cell,Factor1,-0.18622572,B.cell,ACTB
2,B.cell__ACTG1,B.cell,Factor1,-0.05929283,B.cell,ACTG1


### Get top features per factor and for specified threshold

In [45]:
## Get the x% of top features per factor based on the specified threshold (x) in the configuration file

In [46]:
dimensions = unique(feature_weights_long[,c('view', 'variable')])

In [47]:

## Define amont of top genes per fraction 
geneset_oi_pos_per_factor_analyze = feature_weights_long %>% group_by(variable) %>% dplyr::arrange( desc(value),  .by_group = TRUE)  %>% top_frac( as.numeric(top_variable_fraction), value)
geneset_oi_pos_per_factor_analyze$direction = 'positive'

geneset_oi_neg_per_factor_analyze = feature_weights_long %>% group_by(variable) %>% dplyr::arrange(desc(value),  .by_group = TRUE)  %>% top_frac( -as.numeric(top_variable_fraction), value)
geneset_oi_neg_per_factor_analyze$direction = 'negative'

geneset_oi_analyze = rbind(geneset_oi_pos_per_factor_analyze, geneset_oi_neg_per_factor_analyze)
geneset_oi_analyze$fraction = as.numeric(top_variable_fraction)

geneset_oi_per_fraction = geneset_oi_analyze


## Calculate amount of top variables for the selected fraction
amount_geneset_oi_type = geneset_oi_analyze %>% group_by(type, view, variable) %>% dplyr::count()
amount_geneset_oi_type = merge(dimensions, amount_geneset_oi_type, all.x = TRUE)
amount_geneset_oi_type$fraction = as.numeric(top_variable_fraction)


geneset_oi_amounts =amount_geneset_oi_type


In [48]:
## Calculate total amount of features per type
features_per_type = feature_weights_long %>% group_by(type, view, variable) %>% dplyr::count()
colnames(features_per_type) = c('type', 'view', 'variable', 'amount_features')

In [49]:
## Merge top feature amounts and total amounts 
geneset_oi_amounts = merge(  geneset_oi_amounts,features_per_type, all.x = TRUE)

##  Calculate percentages
geneset_oi_amounts$percentage = geneset_oi_amounts$n / geneset_oi_amounts$amount_features

In [50]:
# Set NA when there are no features for this dimension among top %
geneset_oi_amounts[is.na(geneset_oi_amounts)] = 0 

### Prepare heatmap data (based on transformed expression values)

In [51]:
### Subset data on the selected factor (in the configuration)

In [52]:
analysis_data_subset = geneset_oi_per_fraction[geneset_oi_per_fraction$variable %in%  factor_var,]

In [53]:
### Add direction of effect

In [54]:
analysis_data_subset$direction_sign = analysis_data_subset$direction
analysis_data_subset$direction_sign[analysis_data_subset$direction == 'positive'] = '+'
analysis_data_subset$direction_sign[analysis_data_subset$direction == 'negative'] = '-'

In [55]:
head(analysis_data_subset,2)

variable_name,type,variable,value,view,gene,direction,fraction,direction_sign
<chr>,<chr>,<fct>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>
CD14.Mono__PPP1CB,CD14.Mono,Factor2,0.6715288,CD14.Mono,PPP1CB,positive,0.005,+
CD4.TCM__CELF2,CD4.TCM,Factor2,0.6604286,CD4.TCM,CELF2,positive,0.005,+


In [56]:
### Add cell-type/ type column

In [57]:
analysis_data_subset$cell_type = analysis_data_subset$type

In [58]:
### Concat direction + view for faceting on y -axis

In [59]:
analysis_data_subset$facet =  paste0(analysis_data_subset$direction_sign, analysis_data_subset$view)

In [60]:
head(analysis_data_subset,2)

variable_name,type,variable,value,view,gene,direction,fraction,direction_sign,cell_type,facet
<chr>,<chr>,<fct>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
CD14.Mono__PPP1CB,CD14.Mono,Factor2,0.6715288,CD14.Mono,PPP1CB,positive,0.005,+,CD14.Mono,+CD14.Mono
CD4.TCM__CELF2,CD4.TCM,Factor2,0.6604286,CD4.TCM,CELF2,positive,0.005,+,CD4.TCM,+CD4.TCM


In [61]:
## Adjust column names in input data

In [62]:
data_long$expression_value = data_long$value
data_long$variable_name = data_long$variable
data_long$variable = NULL
data_long$value = NULL

In [63]:
head(data_long,2)

,sample_id,type,gene,expression_value,variable_name
,<chr>,<chr>,<chr>,<dbl>,<chr>
1,k10,B.cell,ACTB,0.4307273,B.cell__ACTB
2,k11,B.cell,ACTB,-0.7009514,B.cell__ACTB


In [64]:
### Merge factor values and expression values

In [65]:
heatmap_data = merge(data_long, analysis_data_subset, by.x = c('variable_name'), by.y = c('variable_name'))

In [66]:
nrow(heatmap_data)

[1] 12288

In [67]:
head(heatmap_data,2)

,variable_name,sample_id,type.x,gene.x,expression_value,type.y,variable,value,view,gene.y,direction,fraction,direction_sign,cell_type,facet
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<fct>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>
1,B.cell__HIST1H4C,m23.4,B.cell,HIST1H4C,1.001331,B.cell,Factor2,-0.5710714,B.cell,HIST1H4C,negative,0.005,-,B.cell,-B.cell
2,B.cell__HIST1H4C,k9,B.cell,HIST1H4C,1.150349,B.cell,Factor2,-0.5710714,B.cell,HIST1H4C,negative,0.005,-,B.cell,-B.cell


In [68]:
### Add sample meta-data

In [69]:
heatmap_data = merge(heatmap_data, sample_data, by.x = 'sample_id', by.y = 'sample_id')

In [70]:
head(heatmap_data,2)

,sample_id,variable_name,type.x,gene.x,expression_value,type.y,variable,value,view,gene.y,⋯,cell_type,facet,measurement,classification,delta_ef_value,delta_ef_value_class,CK,CRP,tp_outcome,sc_counts
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<fct>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<int>
1,k1,CD14.Mono__PSME1,CD14.Mono,PSME1,NA,CD14.Mono,Factor2,-0.4925843,CD14.Mono,PSME1,⋯,CD14.Mono,-CD14.Mono,Control,healthy,NA,NA,43,0.4,NA,NA
2,k1,CD4.TCM__HIST1H4C,CD4.TCM,HIST1H4C,NA,CD4.TCM,Factor2,-0.6774162,CD4.TCM,HIST1H4C,⋯,CD4.TCM,-CD4.TCM,Control,healthy,NA,NA,43,0.4,NA,NA


# Heatmap Plot

In [71]:
###Generates heatmap based on configurations

In [72]:
## Specify figure name

In [73]:
figure_name = "FIG05_Heatmap_Feature_Overview"

In [74]:
# Specific Text Descriptions (heatmap) for x and y-axis:
xlabel_heatmap = xlab('Samples') 
ylabel_heatmap = ylab('View')

In [75]:
# Specific Text Descriptions (factor value plot) for x and y-axis:
xlabel_barplot = xlab('Samples') 
ylabel_barplot = ylab('Absolute Factor Value')

In [76]:
## Choose columns for faceting according to parameters specified in config file

In [77]:
## specify faceting for heamtap
heatmap_data$facet2 = heatmap_data[,variable] # x-axis faceting
heatmap_data$facet = paste0(heatmap_data$direction_sign, heatmap_data$view) # y-axis faceting

## specify faceting for factor value plot
analysis_data_subset$facet = paste0(analysis_data_subset$direction_sign,analysis_data_subset$view)
analysis_data_subset$facet2 = 'Absolute Factor Value'

In [78]:
view_var

[1] "CD4.TCM"

In [79]:
## filter data on a specific view if specified in the config file
if(view_var %in% unique(heatmap_data$view)){
    heatmap_data = heatmap_data[heatmap_data$view %in% view_var,]
    analysis_data_subset = analysis_data_subset[analysis_data_subset$view %in% view_var,]
    } else{view_var = ''}

In [80]:
head(heatmap_data,2)

,sample_id,variable_name,type.x,gene.x,expression_value,type.y,variable,value,view,gene.y,⋯,facet,measurement,classification,delta_ef_value,delta_ef_value_class,CK,CRP,tp_outcome,sc_counts,facet2
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<fct>,<dbl>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<chr>,<int>,<chr>
2,k1,CD4.TCM__HIST1H4C,CD4.TCM,HIST1H4C,NA,CD4.TCM,Factor2,-0.6774162,CD4.TCM,HIST1H4C,⋯,-CD4.TCM,Control,healthy,NA,NA,43,0.4,NA,NA,Control
6,k1,CD4.TCM__EEF1A1,CD4.TCM,EEF1A1,NA,CD4.TCM,Factor2,-0.4986455,CD4.TCM,EEF1A1,⋯,-CD4.TCM,Control,healthy,NA,NA,43,0.4,NA,NA,Control


In [81]:
# Plot factor values as barplot

In [82]:
options(repr.plot.width=10, repr.plot.height=25)
factor_value_all_bar = ggplot(data = analysis_data_subset[(analysis_data_subset$variable == factor_var),] , 
                              mapping = aes(x = gene,   y= abs(value)))+
                            plot_config_heatmap_supp  + 
                            geom_bar(stat="identity", width = 0.1) + 
                            coord_flip()+
                            xlabel_barplot +
                            ylabel_barplot +
                            theme( axis.text.y = element_text( hjust = 1, vjust =1))+
                            facet_grid(facet  ~ facet2, scales = 'free', space = 'free', switch = 'y') + theme(strip.background = element_rect(color="black", fill="white", linetype="solid"), strip.placement = "outside", strip.switch.pad.grid = unit(0.5, "cm"))+
                            ggtitle('')


In [83]:
# Example of plot
#factor_value_all_bar

In [84]:
### Generate Heatmap Plot

In [85]:
factor_var

[1] "Factor2"

In [86]:
options(repr.plot.width=width_par, repr.plot.height=height_par)
all_heatmap = ggplot(data = heatmap_data[(heatmap_data$variable == factor_var) ,], # TBD whether to filter on sample subset
                     mapping = aes(sample_id, gene.x,  fill= expression_value)) +
    scale_fill_gradient2(low="#1D2ED8", high="#D8911D", midpoint = 0) + 
    geom_tile() + 
    xlabel_heatmap +
    ylabel_heatmap +
    plot_config_heatmap_supp  + theme(axis.text.x = element_blank()) + 
    facet_grid(facet  ~facet2, scales = 'free', space = 'free',  switch = "y") + theme(strip.background = element_rect(color="black", fill="white", linetype="solid"), strip.placement = "outside", strip.switch.pad.grid = unit(0.4, "cm"), strip.text.y.left = element_text(angle=0))+
    theme( axis.text.y = element_text( hjust = 1, vjust =1)) + 
    ggtitle(paste0('Normalized Expression Values of top ',as.numeric(top_variable_fraction) *100 *2, '% features'))

    

In [87]:
# Example heatmap plot
#all_heatmap

In [88]:
### Combined factor and heatamp plots and save the plot

In [89]:
legend =  get_legend(all_heatmap)

In [90]:
options(repr.plot.width=width_par, repr.plot.height=height_par)
combined = ggarrange(all_heatmap + theme(legend.position = 'none'),
         factor_value_all_bar + 
          theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank(), strip.background.y = element_blank() , strip.text.y = element_blank()),  
         nrow=1, widths = c(4,1), align = 'h')
combined = annotate_figure(combined, right = legend)

In [91]:
pdf(paste0('figures/05_figures/', figure_name,view_var, '_', factor_var, '_',  top_variable_fraction,  mofa_name, '.pdf'), width =width_par, height =height_par)
print(combined)
dev.off()

png 
  2